In [ ]:
# 获取当前notebook的绝对路径
import sys, os
from pathlib import Path
current_path = Path.cwd()  # 或者使用 %pwd magic命令
project_root = current_path.parents[2]  # 如果notebook在project_root/notebooks/下
print("notebook所在的路径：", current_path)
print("project_root：", project_root)
sys.path.append(str(project_root))
import FastAPI_setup

In [2]:
from app.services.state import AgentState, StageEnum, StageStatus, AgentStateHistory, AgentSSEMessage, AgentSSEMessageHistory
from datetime import datetime
# mock AgentState数据
agent_state_1 = AgentState(
    agent_id="123",
    overall_progress=10,
    active_stage=StageEnum.STRUCTURING,
    stage_status=StageStatus.IN_PROGRESS,
    stage_progress=10,
    created_at=datetime.now(),
    updated_at=datetime.now()
)

agent_state_2 = AgentState(
    agent_id="123",
    overall_progress=20,
    active_stage=StageEnum.PLANNING,
    stage_status=StageStatus.IN_PROGRESS,
    stage_progress=20,
    created_at=datetime.now(),
    updated_at=datetime.now()
)

In [3]:
# MOCK AgentSSEMessage数据
agent_sse_message_1 = AgentSSEMessage(
    agent_id="123",
    message="Hello, world!",
    created_at=datetime.now(),
    updated_at=datetime.now()
)

agent_sse_message_2 = AgentSSEMessage(  
    agent_id="123",
    message="Hello, agent!",
    created_at=datetime.now(),
    updated_at=datetime.now()
)

#### 验证agent初始化无错误

In [2]:
from app.core.redis_helper import RedisClient
from app.services.state import AgentState, AgentStateHistory

In [3]:
# project_id = "20563b61-7818-474b-9641-c40757bbd3fe"  # test1
project_id = "cc6bca8a-df5d-43d5-a931-875113df1f50"  # test2
# Project_id = "f6db0cbe-e7af-4300-8335-01ba4ffdbb93"  # small test
# 实例化agent
from app.services.cache import Cache
cache = Cache(project_id)
from app.services.storage import Storage
storage = Storage(project_id)

In [ ]:
await cache.clean_up()

In [ ]:
# 存储状态
await cache.save_agent_state(agent_state_1)

In [ ]:
# 状态和状态历史 获取， 验证是否存储成功
agent_state, agent_state_history =  await cache.get_agent_state()
print(agent_state)
print(agent_state_history)

In [ ]:
# 检查django storage 数据是否存储成功
data = await storage.get_from_django(params={'fields': 'agent_state_history'})
print(data)
print(type(data))

In [ ]:
# 删除缓存
key = cache.get_cache_keys().get('agent_state_history')
from app.core.redis_helper import RedisClient
await RedisClient.delete(key)


In [ ]:
# 缓存数据是否还在？ 理论上应该不在
data = await RedisClient.get(key)
print(data)

In [ ]:
# 检查django storage 数据是否还在？  理论上应该在
data = await storage.get_from_django(params={'fields': 'agent_state_history'})
print(data)
print(type(data))

In [ ]:
# 使用 cache的clean up来删除
await cache.clean_up(['agent_state_history'])

In [ ]:
# 理论上，缓存和storage 数据都不在了
data1 = await RedisClient.get(key)
print(data1)
data2 = await storage.get_from_django(params={'fields': 'agent_state_history'})
print(data2)
print(type(data2))

#### 测试文档

In [4]:
raw_document = {
    'type': 'doc',
    'content': [
        {
            'type': 'paragraph',
            'content': [
                {
                    'type': 'text',
                    'text': 'raw_document_content'
                }
            ]
        }
    ]
}

In [ ]:
data = {
    'key_name': 'raw_document',
    'content': raw_document}
print(type(data))
print(data)

In [ ]:
await cache.save_document('raw_document', raw_document)


In [ ]:
# 缓存数据是否还在？ 理论上应该不在
key = cache.get_cache_keys().get('raw_document')
data = await RedisClient.get(key)
print(data)

In [ ]:
# 删除缓存
key = cache.get_cache_keys().get('raw_document')
from app.core.redis_helper import RedisClient
await RedisClient.delete(key)


In [ ]:
# 缓存数据是否还在？ 理论上应该不在
data = await RedisClient.get(key)
print(data)

In [ ]:
doc = await cache.get_document('raw_document')
print(doc)

In [ ]:
doc_storage = await storage.get_from_django(params={'fields': 'raw_document'})
print(doc_storage)
print(type(doc_storage))
